In [1]:
import os
import numpy as np
import re
import torch
from torchvision import transforms
from PIL import Image, UnidentifiedImageError
import numpy as np
import pandas as pd


In [ ]:

## 원본 사이즈 저장. 수정/원본 비율저장 코드 제작 파일 이름이랑 같이 저장
file_name=[]
original_width=[]
original_height=[]
resize_width_ratio=[]
resize_height_ratio=[]
padding_x=[]
padding_y=[]

transform = transforms.Compose([
    transforms.Resize((640, 640)),  # 이미지 크기 조정
    transforms.CenterCrop(640),  # 이미지 중앙을 기준으로 640x640 크기로 잘라냄
    transforms.ToTensor(),  # PIL 이미지나 ndarray를 PyTorch 텐서로 변환
])

main_dir='.\\원천데이터'

emo_list=os.listdir(main_dir)

emo_list
for emo in emo_list: # 감성 리스트
    file_folder_path=os.path.join(main_dir,emo) # main_dir + 감성 
    print(file_folder_path)


    match=re.split('_', emo) # 폴더명에서 감성 (기쁨, 슬픔 ) 만 가져오기
    match=match[1]

    file_list=os.listdir(file_folder_path) # 파일이 들어있는 폴더 경로
    print(file_list)

    for file in file_list:
        origin_file_adress=os.path.join(file_folder_path,file) #파일 경로

        file_name.append(file) # 파일명 저장

        if os.path.exists(origin_file_adress):
            try:
                image = Image.open(origin_file_adress)
                image_tensor = transform(image)
                save_dir = os.path.join('./수정본/', match, file)

                if not os.path.exists(save_dir):
                    os.makedirs(os.path.dirname(save_dir), exist_ok=True)

                torch.save(image_tensor, save_dir)
                print(save_dir)

            except (OSError, UnidentifiedImageError) as e:
                if 'image file is truncated' in str(e):
                    print(f"Error loading image: {origin_file_adress}. Skipping to the next image.")
                else:
                    raise e

In [5]:
# 데이터 프레임 생성을 위한 딕셔너리 생성
dataf={}
dataf['file_name']=file_name
dataf['original_width']=original_width
dataf['original_height']=original_height
dataf['padding_x']=padding_x
dataf['padding_y']=padding_y
dataf['resize_width_ratio']=resize_width_ratio
dataf['resize_height_ratio']=resize_height_ratio

# 원본 사진 크기와  640 비율 데이터 프레임 생성
df = pd.DataFrame(dataf)
df.to_csv('./orizin_resize.csv',index=False)